<a href="https://colab.research.google.com/github/mkufel/CryptoTrader/blob/master/CryptoTrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Dataset https://www.kaggle.com/sudalairajkumar/cryptocurrencypricehistory#ethereum_price.csv

In [68]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    data_path = '/content/drive/My Drive/ID2223/crypto_data/'
    course_dir_path = '/content/drive/My Drive/ID2223'
except:
  print("Failed to connect to Drive")

Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [0]:
currencies = ["bitcoin", "bitconnect", "dash", "ethereum", "ethereum_classic", "iota", "litecoin", "monero", "nem", "neo", "numeraire", "omisego", "ripple", "stratis" , "waves"]

In [71]:
df_test = pd.read_csv(f"{data_path}bitcoin_price.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
df_test.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,Date,Open,High,Low,Close,Volume,Market Cap
1,"Feb 20, 2018",11231.80,11958.50,11231.80,11403.70,"9,926,540,000","189,536,000,000"
2,"Feb 19, 2018",10552.60,11273.80,10513.20,11225.30,"7,652,090,000","178,055,000,000"
3,"Feb 18, 2018",11123.40,11349.80,10326.00,10551.80,"8,744,010,000","187,663,000,000"
4,"Feb 17, 2018",10207.50,11139.50,10149.40,11112.70,"8,660,880,000","172,191,000,000"


In [72]:
# Get the date column for further joins, reverse the order
all_data = pd.DataFrame(data=df_test['Date'])[::-1]

for currency in currencies:
  # Read a file for the given currency
  df = pd.read_csv(f'{data_path}{currency}_price.csv', names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
  df = df.iloc[1:]

  # Rename columns adding currency names in headers
  df.rename(columns={
      'Open': f'Open_{currency}',
      'High': f'High_{currency}',
      'Low': f'Low_{currency}',
      'Close': f'Close_{currency}',
      'Volume': f'Volume_{currency}',
      'Market Cap': f'Market_cap_{currency}'},
      inplace=True)
  
  # Merge the renamed data frame with all_data on the Date column
  all_data = pd.merge(all_data, df, on='Date')

print(all_data.head(5))

           Date Open_bitcoin High_bitcoin Low_bitcoin Close_bitcoin Volume_bitcoin Market_cap_bitcoin Open_bitconnect High_bitconnect Low_bitconnect Close_bitconnect Volume_bitconnect Market_cap_bitconnect Open_dash High_dash Low_dash Close_dash Volume_dash Market_cap_dash Open_ethereum High_ethereum Low_ethereum Close_ethereum Volume_ethereum Market_cap_ethereum Open_ethereum_classic High_ethereum_classic Low_ethereum_classic Close_ethereum_classic Volume_ethereum_classic Market_cap_ethereum_classic Open_iota High_iota  Low_iota Close_iota Volume_iota Market_cap_iota Open_litecoin High_litecoin Low_litecoin  ...   Low_nem Close_nem Volume_nem Market_cap_nem Open_neo High_neo Low_neo Close_neo  Volume_neo Market_cap_neo Open_numeraire High_numeraire Low_numeraire Close_numeraire Volume_numeraire Market_cap_numeraire Open_omisego High_omisego Low_omisego Close_omisego Volume_omisego Market_cap_omisego Open_ripple High_ripple Low_ripple Close_ripple Volume_ripple Market_cap_ripple Open_s

In [0]:
TARGET_CURRENCY = 'waves'
SEQ_LEN_PAST = 3
SEQ_LEN_FUT = 1

In [74]:
all_data['Future'] = all_data[f"Close_{TARGET_CURRENCY}"].shift(-SEQ_LEN_FUT)
all_data.dropna(inplace=True)
all_data.tail()

,Date,Open_bitcoin,High_bitcoin,Low_bitcoin,Close_bitcoin,Volume_bitcoin,Market_cap_bitcoin,Open_bitconnect,High_bitconnect,Low_bitconnect,Close_bitconnect,Volume_bitconnect,Market_cap_bitconnect,Open_dash,High_dash,Low_dash,Close_dash,Volume_dash,Market_cap_dash,Open_ethereum,High_ethereum,Low_ethereum,Close_ethereum,Volume_ethereum,Market_cap_ethereum,Open_ethereum_classic,High_ethereum_classic,Low_ethereum_classic,Close_ethereum_classic,Volume_ethereum_classic,Market_cap_ethereum_classic,Open_iota,High_iota,Low_iota,Close_iota,Volume_iota,Market_cap_iota,Open_litecoin,High_litecoin,Low_litecoin,...,Close_nem,Volume_nem,Market_cap_nem,Open_neo,High_neo,Low_neo,Close_neo,Volume_neo,Market_cap_neo,Open_numeraire,High_numeraire,Low_numeraire,Close_numeraire,Volume_numeraire,Market_cap_numeraire,Open_omisego,High_omisego,Low_omisego,Close_omisego,Volume_omisego,Market_cap_omisego,Open_ripple,High_ripple,Low_ripple,Close_ripple,Volume_ripple,Market_cap_ripple,Open_stratis,High_stratis,Low_stratis,Close_stratis,Volume_stratis,Market_cap_stratis,Open_waves,High_waves,Low_waves,Close_waves,Volume_waves,Market_cap_waves,Future
216,"Feb 15, 2018",9488.32,10234.80,9395.58,10166.40,"9,062,540,000","160,025,000,000",3.86,4.30,3.70,4.04,"157,578","35,618,900",679.79,705.67,667.21,703.86,"116,822,000","5,359,910,000",923.73,951.95,911.11,936.98,"2,900,100,000","90,173,300,000",34.83,35.65,32.83,34.53,"635,228,000","3,477,970,000",2.03,2.17,2.00,2.13,"65,315,300","5,635,360,000",212.35,237.88,211.17,...,0.561359,"67,295,600","5,121,400,000",122.00,125.76,118.79,123.29,"218,616,000","7,930,300,000",22.04,23.17,21.13,22.16,"186,324","28,493,800",15.63,16.71,14.99,16.67,"54,142,700","1,594,790,000",1.14,1.16,1.11,1.15,"1,188,530,000","44,433,600,000",9.26,9.76,9.08,9.54,"23,848,900","914,277,000",6.42,8.07,6.38,8.07,"61,670,200","641,992,000",7.90
217,"Feb 16, 2018",10135.70,10324.10,9824.82,10233.90,"7,296,160,000","170,960,000,000",4.06,4.19,3.87,4.13,"149,639","37,459,600",702.53,722.02,684.28,698.86,"100,343,000","5,540,570,000",934.79,950.00,917.85,944.21,"2,369,450,000","91,272,100,000",34.43,35.34,32.90,35.03,"456,754,000","3,439,180,000",2.12,2.18,2.00,2.10,"44,770,200","5,894,520,000",225.50,236.09,212.33,...,0.568887,"90,028,500","5,040,780,000",123.02,130.62,119.94,128.85,"194,878,000","7,996,380,000",22.06,22.68,20.55,21.25,"106,464","28,525,200",16.67,18.23,16.03,18.17,"81,337,500","1,701,370,000",1.14,1.15,1.11,1.14,"568,339,000","44,663,300,000",9.50,9.79,9.00,9.56,"14,430,300","938,100,000",8.00,8.36,7.49,7.90,"44,143,700","800,093,000",7.84
218,"Feb 17, 2018",10207.50,11139.50,10149.40,11112.70,"8,660,880,000","172,191,000,000",4.08,4.38,4.01,4.31,"185,105","37,708,600",699.13,748.89,693.64,742.16,"116,106,000","5,515,090,000",944.75,976.60,940.75,974.12,"2,525,720,000","92,264,000,000",34.97,35.40,34.38,34.99,"418,534,000","3,493,650,000",2.10,2.21,2.08,2.20,"42,149,700","5,842,140,000",229.50,233.45,225.60,...,0.598880,"147,788,000","5,091,170,000",131.03,139.80,129.42,136.75,"286,385,000","8,516,620,000",21.24,23.32,20.99,23.25,"212,041","27,463,300",18.17,18.64,17.69,18.29,"59,295,600","1,853,770,000",1.14,1.22,1.13,1.20,"1,186,730,000","44,370,200,000",9.53,9.95,9.42,9.76,"19,415,600","940,978,000",7.88,7.96,7.63,7.84,"34,458,200","788,332,000",7.54
219,"Feb 18, 2018",11123.40,11349.80,10326.00,10551.80,"8,744,010,000","187,663,000,000",4.34,4.34,3.66,3.70,"150,823","40,037,400",744.09,747.42,673.57,689.28,"96,752,600","5,871,020,000",973.35,982.93,915.45,923.92,"2,567,290,000","95,077,100,000",35.01,35.08,31.52,34.88,"456,829,000","3,498,390,000",2.20,2.21,1.98,1.98,"43,653,800","6,121,620,000",230.51,232.64,214.46,...,0.530407,"95,781,300","5,357,680,000",136.40,137.95,124.32,127.38,"207,624,000","8,866,100,000",22.69,23.78,20.43,20.43,"145,246","29,298,400",18.32,18.41,16.35,16.66,"42,287,800","1,869,770,000",1.20,1.21,1.09,1.12,"1,113,650,000","46,997,500,000",9.76,9.78,8.82,9.26,"14,139,400","963,499,000",7.84,8.09,7

In [75]:
# Add a column of target labels, if the rate rose in SEQ_LEN_FUT: 1, if fell: 0
all_data['Target'] = list(map(lambda past, future: int(float(future) > float(past)), all_data[f'Close_{TARGET_CURRENCY}'], all_data['Future']))
all_data[[f'Close_{TARGET_CURRENCY}', 'Future', 'Target']].head(5)

,Close_waves,Future,Target
0,2.43,2.04,0
1,2.04,1.97,0
2,1.97,2.52,1
3,2.52,2.83,1
4,2.83,2.64,0


In [76]:
from datetime import datetime
import time

def transform_date(date):
  return str(time.mktime(datetime.strptime(date, '%b %d, %Y').timetuple()))

all_data['Date'] = all_data['Date'].apply(transform_date)

all_data['Date'].head(5)

0    1499990400.0
1    1500076800.0
2    1500163200.0
3    1500249600.0
4    1500336000.0
Name: Date, dtype: object

In [77]:
cols = all_data.columns
all_data[cols] = all_data[cols].replace({',': ''}, regex=True)

all_data.head(220)

,Date,Open_bitcoin,High_bitcoin,Low_bitcoin,Close_bitcoin,Volume_bitcoin,Market_cap_bitcoin,Open_bitconnect,High_bitconnect,Low_bitconnect,Close_bitconnect,Volume_bitconnect,Market_cap_bitconnect,Open_dash,High_dash,Low_dash,Close_dash,Volume_dash,Market_cap_dash,Open_ethereum,High_ethereum,Low_ethereum,Close_ethereum,Volume_ethereum,Market_cap_ethereum,Open_ethereum_classic,High_ethereum_classic,Low_ethereum_classic,Close_ethereum_classic,Volume_ethereum_classic,Market_cap_ethereum_classic,Open_iota,High_iota,Low_iota,Close_iota,Volume_iota,Market_cap_iota,Open_litecoin,High_litecoin,Low_litecoin,...,Volume_nem,Market_cap_nem,Open_neo,High_neo,Low_neo,Close_neo,Volume_neo,Market_cap_neo,Open_numeraire,High_numeraire,Low_numeraire,Close_numeraire,Volume_numeraire,Market_cap_numeraire,Open_omisego,High_omisego,Low_omisego,Close_omisego,Volume_omisego,Market_cap_omisego,Open_ripple,High_ripple,Low_ripple,Close_ripple,Volume_ripple,Market_cap_ripple,Open_stratis,High_stratis,Low_stratis,Close_stratis,Volume_stratis,Market_cap_stratis,Open_waves,High_waves,Low_waves,Close_waves,Volume_waves,Market_cap_waves,Future,Target
0,1499990400.0,2360.59,2363.25,2183.22,2233.34,882503000,38823100000,57.60,58.57,53.94,55.53,3445520,398102000,171.48,171.91,156.49,161.08,26831900,1273130000,209.53,211.10,187.37,199.66,749382000,19541300000,18.41,18.60,16.85,17.55,215528000,1722240000,0.212999,0.214112,0.161877,0.175350,3347860,592038000,46.36,46.48,40.80,...,2077230,1104370000,6.27,6.29,5.42,5.54,12351000,313433000,26.80,27.67,21.18,23.24,424550,32787700,0.534731,0.775406,0.500232,0.582480,750698,32380400,0.200819,0.201156,0.183369,0.190769,97037600,7689650000,3.66,3.66,2.99,3.29,9898950,360072000,2.89,2.89,2.33,2.43,2439500,289495000,2.04,0
1,1500076800.0,2230.12,2231.14,1990.41,1998.86,993608000,36681300000,55.45,55.55,44.57,44.57,2687210,383760000,161.28,163.44,139.91,140.14,30810000,1197760000,199.71,200.16,170.66,170.66,810452000,18630200000,17.60,17.60,14.38,14.77,243159000,1646950000,0.175237,0.176099,0.147933,0.158688,3913220,487076000,42.81,42.81,37.78,...,3213760,1057430000,5.55,5.55,4.98,5.03,11294200,277400000,23.20,23.24,18.69,19.05,269653,28383800,0.586446,0.652156,0.433351,0.433351,1552730,39380400,0.190976,0.191167,0.167118,0.170589,131779000,7312750000,3.29,3.29,2.87,3.19,5250730,324366000,2.43,2.43,1.98,2.04,2860750,242705000,1.97,0
2,1500163200.0,1991.98,2058.77,1843.03,1929.82,1182870000,32767600000,44.22,50.56,29.87,48.05,2100690,260770000,139.65,143.54,119.17,132.64,35385900,1037360000,169.57,172.01,133.72,157.36,1516890000,15823100000,14.71,15.36,12.25,14.18,249231000,1376490000,0.157961,0.185445,0.155173,0.180978,4178170,439057000,38.92,41.97,36.38,...,2583610,999834000,5.03,5.27,4.83,5.04,9498420,251399000,18.99,20.06,16.54,17.49,213544,23236900,0.431081,0.494342,0.319695,0.384906,1743630,42380400,0.169480,0.172575,0.119770,0.148056,257944000,6489640000,3.15,3.26,2.50,2.88,5474900,310564000,2.01,2.06,1.77,1.97,1677160,201211000,2.52,1
3,1500249600.0,1932.62,2230.49,1932.62,2228.41,1201760000,31795000000,47.74,55.83,47.74,55.49,3778790,281529000,132.34,154.91,132.34,153.85,32658200,983325000,159.99,194.75,158.04,193.42,1669500000,14932700000,14.40,15.69,13.80,15.50,300782000,1348150000,0.180717,0.278088,0.179114,0.263105,15254300,502309000,41.19,43.07,40.17,...,4458730,876887000,5.06,5.85,5.03,5.77,15013000,253054000,17.46,22.79,17.46,22.69,272026,21363400,0.381684,0.598633,0.379345,0.582786,4273090,37524100,0.150146,0.179604,0.141425,0.177889,312685000,5749320000,2.87,3.90,2.86,3.88,8947560,282410000,1.97,2.52,1.94,2.52,1909110,196939000,2.83,1
4,1500336000.0,2233.52,2387.61,2164.77,2318.88,1512450000,36749400000,55.72,59.98,53.81,58.29,3540910,329330000,154.50,165.70,146.25,158.07,28103500,1148330000,195.03,267.99,182.06,234.39,2709260000,18207400000,15.51,17.06,14.79,16.40,282383000,1451740000,0.264569,0.309577,0.248754,0.267340,11433600,735378000,42.62,45.33,41.47,...,5351550,1222820000,5.77,6.29,5.51,6.02,18101700,2

In [78]:
  # Validation set last 15pc
last_15pc = all_data['Date'][int(0.85*len(all_data.index.values))]
last_15pc

'1516147200.0'

In [79]:
val_df = all_data[(all_data['Date'] >= last_15pc)]

# all_data.loc[all_data['Date'] >= last_15pc] = val_df
print(f"Validation set shape: {val_df.shape}")

# all_data.loc[all_data['Date'] < last_15pc] = train_df
train_df = all_data[(all_data['Date'] < last_15pc)]
print(f"Training set shape: {train_df.shape}")
train_df.head(186)
# val_df.head()


Validation set shape: (34, 93)
Training set shape: (187, 93)


,Date,Open_bitcoin,High_bitcoin,Low_bitcoin,Close_bitcoin,Volume_bitcoin,Market_cap_bitcoin,Open_bitconnect,High_bitconnect,Low_bitconnect,Close_bitconnect,Volume_bitconnect,Market_cap_bitconnect,Open_dash,High_dash,Low_dash,Close_dash,Volume_dash,Market_cap_dash,Open_ethereum,High_ethereum,Low_ethereum,Close_ethereum,Volume_ethereum,Market_cap_ethereum,Open_ethereum_classic,High_ethereum_classic,Low_ethereum_classic,Close_ethereum_classic,Volume_ethereum_classic,Market_cap_ethereum_classic,Open_iota,High_iota,Low_iota,Close_iota,Volume_iota,Market_cap_iota,Open_litecoin,High_litecoin,Low_litecoin,...,Volume_nem,Market_cap_nem,Open_neo,High_neo,Low_neo,Close_neo,Volume_neo,Market_cap_neo,Open_numeraire,High_numeraire,Low_numeraire,Close_numeraire,Volume_numeraire,Market_cap_numeraire,Open_omisego,High_omisego,Low_omisego,Close_omisego,Volume_omisego,Market_cap_omisego,Open_ripple,High_ripple,Low_ripple,Close_ripple,Volume_ripple,Market_cap_ripple,Open_stratis,High_stratis,Low_stratis,Close_stratis,Volume_stratis,Market_cap_stratis,Open_waves,High_waves,Low_waves,Close_waves,Volume_waves,Market_cap_waves,Future,Target
0,1499990400.0,2360.59,2363.25,2183.22,2233.34,882503000,38823100000,57.60,58.57,53.94,55.53,3445520,398102000,171.48,171.91,156.49,161.08,26831900,1273130000,209.53,211.10,187.37,199.66,749382000,19541300000,18.41,18.60,16.85,17.55,215528000,1722240000,0.212999,0.214112,0.161877,0.175350,3347860,592038000,46.36,46.48,40.80,...,2077230,1104370000,6.27,6.29,5.42,5.54,12351000,313433000,26.80,27.67,21.18,23.24,424550,32787700,0.534731,0.775406,0.500232,0.582480,750698,32380400,0.200819,0.201156,0.183369,0.190769,97037600,7689650000,3.66,3.66,2.99,3.29,9898950,360072000,2.89,2.89,2.33,2.43,2439500,289495000,2.04,0
1,1500076800.0,2230.12,2231.14,1990.41,1998.86,993608000,36681300000,55.45,55.55,44.57,44.57,2687210,383760000,161.28,163.44,139.91,140.14,30810000,1197760000,199.71,200.16,170.66,170.66,810452000,18630200000,17.60,17.60,14.38,14.77,243159000,1646950000,0.175237,0.176099,0.147933,0.158688,3913220,487076000,42.81,42.81,37.78,...,3213760,1057430000,5.55,5.55,4.98,5.03,11294200,277400000,23.20,23.24,18.69,19.05,269653,28383800,0.586446,0.652156,0.433351,0.433351,1552730,39380400,0.190976,0.191167,0.167118,0.170589,131779000,7312750000,3.29,3.29,2.87,3.19,5250730,324366000,2.43,2.43,1.98,2.04,2860750,242705000,1.97,0
2,1500163200.0,1991.98,2058.77,1843.03,1929.82,1182870000,32767600000,44.22,50.56,29.87,48.05,2100690,260770000,139.65,143.54,119.17,132.64,35385900,1037360000,169.57,172.01,133.72,157.36,1516890000,15823100000,14.71,15.36,12.25,14.18,249231000,1376490000,0.157961,0.185445,0.155173,0.180978,4178170,439057000,38.92,41.97,36.38,...,2583610,999834000,5.03,5.27,4.83,5.04,9498420,251399000,18.99,20.06,16.54,17.49,213544,23236900,0.431081,0.494342,0.319695,0.384906,1743630,42380400,0.169480,0.172575,0.119770,0.148056,257944000,6489640000,3.15,3.26,2.50,2.88,5474900,310564000,2.01,2.06,1.77,1.97,1677160,201211000,2.52,1
3,1500249600.0,1932.62,2230.49,1932.62,2228.41,1201760000,31795000000,47.74,55.83,47.74,55.49,3778790,281529000,132.34,154.91,132.34,153.85,32658200,983325000,159.99,194.75,158.04,193.42,1669500000,14932700000,14.40,15.69,13.80,15.50,300782000,1348150000,0.180717,0.278088,0.179114,0.263105,15254300,502309000,41.19,43.07,40.17,...,4458730,876887000,5.06,5.85,5.03,5.77,15013000,253054000,17.46,22.79,17.46,22.69,272026,21363400,0.381684,0.598633,0.379345,0.582786,4273090,37524100,0.150146,0.179604,0.141425,0.177889,312685000,5749320000,2.87,3.90,2.86,3.88,8947560,282410000,1.97,2.52,1.94,2.52,1909110,196939000,2.83,1
4,1500336000.0,2233.52,2387.61,2164.77,2318.88,1512450000,36749400000,55.72,59.98,53.81,58.29,3540910,329330000,154.50,165.70,146.25,158.07,28103500,1148330000,195.03,267.99,182.06,234.39,2709260000,18207400000,15.51,17.06,14.79,16.40,282383000,1451740000,0.264569,0.309577,0.248754,0.267340,11433600,735378000,42.62,45.33,41.47,...,5351550,1222820000,5.77,6.29,5.51,6.02,18101700,2

In [0]:
from sklearn import preprocessing
from collections import deque
import random

def preprocess(df):
  df.drop('Future', 1)
  
  for col in df.columns:
    if col != 'Target':
      df[col] = df[col].astype(float).pct_change()
      # print(df[df.isnull().any(axis=1)])
      # df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values)

  df.dropna(inplace=True)

  # Generate sequences
  seq_data = []
  prev_days = deque(maxlen=SEQ_LEN_PAST)

  for i in df.values:
    prev_days.append([n for n in i[:-1]])
    if len(prev_days) == SEQ_LEN_PAST:
      seq_data.append([np.array(prev_days), i[-1]])

  random.shuffle(seq_data)

  # Balance the data
  buys = []
  sells = []

  for seq, target in seq_data:
    if target == 0:
      sells.append([seq, target])
    elif target == 1:
      buys.append([seq, target])

  lower = min(len(buys), len(sells))

  buys = buys[:lower]
  sells = sells[:lower]

  seq_data = buys + sells
  random.shuffle(seq_data)
  
  X = []
  Y = []

  for seq, target in seq_data:
    X.append(seq)
    Y.append(target)
  
  return np.array(X), Y

In [81]:
pd.set_option('display.expand_frame_repr', False)

features_train, labels_train = preprocess(train_df.copy())
features_val, labels_val = preprocess(val_df.copy())

print(f'Training features length: {len(features_train)}, labels length {len(labels_train)}')
print(f'Validation features length: {len(features_val)}, labels length {len(labels_val)}')

# print(features_val)
# print(labels_val)

Training features length: 184, labels length 184
Validation features length: 28, labels length 28


In [82]:
features_val, labels_val = preprocess(val_df.copy())
# print(f'Validation features length: {len(features_val)}, labels length {len(labels_val)}')

test_features = features_val[-1:]
len(test_features)

1

In [0]:
import time
EPOCHS = 10
BATCH_SIZE = 3

In [84]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint


model = Sequential()
model.add(LSTM(128, input_shape=(features_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(features_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(features_train.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

tensorboard = TensorBoard(log_dir='logs/')

filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("{}/trader_models/{}.model".format(course_dir_path, filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

history = model.fit(features_train, labels_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(features_val, labels_val), callbacks=[tensorboard, checkpoint])


Train on 184 samples, validate on 28 samples
Epoch 1/10
184/184 [==============================] - 15s 83ms/sample - loss: 0.9907 - acc: 0.5163 - val_loss: 0.6934 - val_acc: 0.5000
Epoch 2/10
184/184 [==============================] - 2s 12ms/sample - loss: 0.8465 - acc: 0.5435 - val_loss: 0.7092 - val_acc: 0.5000
Epoch 3/10
184/184 [==============================] - 2s 12ms/sample - loss: 0.7634 - acc: 0.5978 - val_loss: 0.6841 - val_acc: 0.5357
Epoch 4/10
184/184 [==============================] - 2s 11ms/sample - loss: 0.6661 - acc: 0.6359 - val_loss: 0.7574 - val_acc: 0.4286
Epoch 5/10
184/184 [==============================] - 2s 10ms/sample - loss: 0.7892 - acc: 0.5870 - val_loss: 0.6955 - val_acc: 0.5000
Epoch 6/10
184/184 [==============================] - 2s 12ms/sample - loss: 0.6543 - acc: 0.6359 - val_loss: 0.7835 - val_acc: 0.4643
Epoch 7/10
184/184 [==============================] - 2s 11ms/sample - loss: 0.6262 - acc: 0.6304 - val_loss: 0.8723 - val_acc: 0.5000
Epoch 8/1

In [85]:
# Get the date column for further joins, reverse the order
all_data = pd.DataFrame(data=df_test['Date'])[::-1]

for currency in currencies:
  # Read a file for the given currency
  df = pd.read_csv(f'{data_path}{currency}_price.csv', names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
  df = df.iloc[1:]

  # Rename columns adding currency names in headers
  df.rename(columns={
      'Open': f'Open_{currency}',
      'High': f'High_{currency}',
      'Low': f'Low_{currency}',
      'Close': f'Close_{currency}',
      'Volume': f'Volume_{currency}',
      'Market Cap': f'Market_cap_{currency}'},
      inplace=True)
  
  # Merge the renamed data frame with all_data on the Date column
  all_data = pd.merge(all_data, df, on='Date')

print(all_data.head(5))

# model.predict()

           Date Open_bitcoin High_bitcoin Low_bitcoin Close_bitcoin Volume_bitcoin Market_cap_bitcoin Open_bitconnect High_bitconnect Low_bitconnect Close_bitconnect Volume_bitconnect Market_cap_bitconnect Open_dash High_dash Low_dash Close_dash Volume_dash Market_cap_dash Open_ethereum High_ethereum Low_ethereum Close_ethereum Volume_ethereum Market_cap_ethereum Open_ethereum_classic High_ethereum_classic Low_ethereum_classic Close_ethereum_classic Volume_ethereum_classic Market_cap_ethereum_classic Open_iota High_iota  Low_iota Close_iota Volume_iota Market_cap_iota Open_litecoin High_litecoin Low_litecoin  ...   Low_nem Close_nem Volume_nem Market_cap_nem Open_neo High_neo Low_neo Close_neo  Volume_neo Market_cap_neo Open_numeraire High_numeraire Low_numeraire Close_numeraire Volume_numeraire Market_cap_numeraire Open_omisego High_omisego Low_omisego Close_omisego Volume_omisego Market_cap_omisego Open_ripple High_ripple Low_ripple Close_ripple Volume_ripple Market_cap_ripple Open_s

In [86]:
# print(input_pred)
model.predict(test_features)

array([[0.613079, 0.386921]], dtype=float32)